In [1]:
import pandas as pd
import numpy as np
train = pd.read_csv("/Users/xinchengzhu/Downloads/train.csv")

In [3]:
train = train[train['MISSING_DATA'] == False]

In [4]:
train = train[train['POLYLINE'].map(len) > 1]

In [5]:
train = train[['POLYLINE']]

In [46]:
train_1 = train.iloc[1:10001]
train_1 = train_1[train_1['POLYLINE'] != '[]']
train_1

,POLYLINE
1,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."
5,"[[-8.615502,41.140674],[-8.614854,41.140926],[..."
...,...
9996,"[[-8.583129,41.164731],[-8.583129,41.164785],[..."
9997,"[[-8.600103,41.18274],[-8.599806,41.182722],[-..."
9998,"[[-8.5878,41.174568],[-8.587782,41.174595],[-8..."
9999,"[[-8.613144,41.166774],[-8.613333,41.166675],[..."


In [7]:
a = train_1['POLYLINE'].iloc[0]
a = a[2:-2]
b = a.split('],[')
new = []
for item in b:
    new.append(item.split(','))
for i in range(len(new)):
    new[i] = [float(j) for j in new[i]]
print(type(new[0][0]))

<class 'float'>


In [47]:
for i in range(len(train_1)):
    a = train_1['POLYLINE'].iloc[i]
    a = a[2:-2]
    new = []
    for item in b:
        new.append(item.split(','))
    for k in range(len(new)):
        new[k] = [float(j) for j in new[k]]
    train_1['POLYLINE'].iloc[i] = new

In [58]:
train_1["POLYLINE"].iloc(0)[0][0]

[-8.641665, 41.156604]

In [62]:
max_latitude = train_1["POLYLINE"].iloc(0)[0][0][0]
min_latitude = train_1["POLYLINE"].iloc(0)[0][0][0]
max_longtitude = train_1["POLYLINE"].iloc(0)[0][0][1]
min_longtitude = train_1["POLYLINE"].iloc(0)[0][0][1]
for i in range(len(train_1)):
    for cord in train_1["POLYLINE"].iloc(0)[i]:
        max_latitude = max(max_latitude,cord[0])
        max_longtitude = max(max_longtitude,cord[1])
        min_latitude = min(max_latitude,cord[0])
        min_longtitude = min(max_longtitude,cord[1])
print(max_longtitude,max_latitude,min_latitude,min_longtitude)

41.174226 -8.606322 -8.620776 41.155902


<class 'str'>
